In [1]:
import torch
import torchvision
import pandas as pd

In [2]:
ls

Train Transformer 1.ipynb  dataloader.py
__pycache__/


In [3]:
from dataloader import *

In [4]:
comment_df = pd.read_csv("../data/attack_annotated_comments.tsv", sep ='\t')
body_df = pd.read_csv("../data/fake_news_bodies.csv")
stance_df = pd.read_csv("../data/fake_news_stances.csv")
vocab = Vocabulary([comment_df["comment"], body_df["articleBody"], stance_df["Headline"]])
annotation_df = pd.read_csv("../data/attack_annotations.tsv",  sep='\t')

wiki_dataset = WikiDataset(comment_df, annotation_df, vocab)
fake_news_dataset = FakeNewsDataset(body_df, stance_df, vocab)

In [5]:
comment_df.head()

,rev_id,comment,year,logged_in,ns,sample,split
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train
3,89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev
4,93890,This page will need disambiguation.,2002,True,article,random,train


In [6]:
body_df.head()

,Body ID,articleBody,sentence_as_idx
0,0,A small meteorite crashed into a wooded area i...,"[2, 126, 2627, 7088, 36713, 1541, 126, 64723, ..."
1,4,Last week we hinted at what was to come as Ebo...,"[2, 978, 9959, 301, 27551, 181, 184, 243, 21, ..."
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...,"[2, 453967, 6041, 843, 446, 1764, 126, 28996, ..."
3,6,"Posting photos of a gun-toting child online, I...","[2, 6277, 11409, 13, 126, 185851, 6544, 7820, ..."
4,7,At least 25 suspected Boko Haram insurgents we...,"[2, 181, 425, 2299, 27854, 420528, 12253, 1034..."


In [7]:
stance_df.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [39]:
stance_df['Stance'].unique()

array(['unrelated', 'agree', 'disagree', 'discuss'], dtype=object)

In [38]:
annotation_df.head()

,rev_id,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
0,37675,1362,0.0,0.0,0.0,0.0,0.0
1,37675,2408,0.0,0.0,0.0,0.0,0.0
2,37675,1493,0.0,0.0,0.0,0.0,0.0
3,37675,1439,0.0,0.0,0.0,0.0,0.0
4,37675,170,0.0,0.0,0.0,0.0,0.0


In [50]:
fake_news_dataset.y.mean(axis=0)

array([0.73130953, 0.07360122, 0.01680941, 0.17827984])

In [53]:
np.array([np.array(x) for x in fake_news_dataset.x])

array([array([     2,   9064,    543,   3470,  15547,     60,    181,    425,
       452490, 473495,   5122,  21717,   5557,    357,  68000,  16242,
        23610,   2110,   9064,   1329,      3,      2,  12701,  70946,
            5,  41000,     10,  96790,    748,  88758, 241925,      5,
           80,  24993,     21,   2433,  15651,   6485,   2316, 455896,
           67, 461037,   2316,  15887, 458699,  12701,  70946,      5,
        41000,     10,  96790,   3261,    622,    342,  23209, 285237,
         3021,     16,  70667,     82,  70848, 457707,    349,      5,
          221,     13,     10,    544,  53088, 454830,     67,   1309,
          585,   4478,    434,      6,    297, 265230,     16,  16233,
            6,    148,    290,     23, 241925,    831,    173,   3011,
        72889,     91,     10,  10021,   4660,  56424, 457714, 454838,
       454839,     16,   1775,  19455,   4660,    434,   2883,    513,
        60239,    342,     10,  44587,     16,      9,     67, 143774,

In [60]:
annotation_df['attack']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
          ... 
1365187    1.0
1365188    0.0
1365189    0.0
1365190    0.0
1365191    0.0
1365192    0.0
1365193    0.0
1365194    0.0
1365195    0.0
1365196    0.0
1365197    0.0
1365198    0.0
1365199    0.0
1365200    1.0
1365201    0.0
1365202    1.0
1365203    0.0
1365204    0.0
1365205    0.0
1365206    0.0
1365207    0.0
1365208    0.0
1365209    0.0
1365210    0.0
1365211    0.0
1365212    0.0
1365213    0.0
1365214    0.0
1365215    0.0
1365216    0.0
Name: attack, Length: 1365217, dtype: float64

In [68]:
a = np.array([1, 1, 0, 1, 0])
n = 2 # num classes
b = np.array([np.array([int(i == j) for i in range(n)]) for j in a])
b

array([[0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0]])